In [ ]:
from web3 import HTTPProvider, Web3
import json
import os

In [ ]:
RPC_HOST = 'localhost'
RPC_PORT = '8545'
W3 = Web3(HTTPProvider('http://' + RPC_HOST+':'+RPC_PORT))
MEETUP_GROUP_FACTORY_ADDRESS = Web3.toChecksumAddress('0x4507e90832a4a17eb15b107d817bb0ceb3430849')
ENS_FACTORY_ADDRESS = Web3.toChecksumAddress('0xb517c0842a63995cea94929a5160386dffa5cc58')

In [ ]:
def load_interface(interface_name):
    with open('../build/contracts/'+ interface_name + '.json', 'r') as f:
        interface = json.load(f)
    return interface

In [ ]:
meetup_group_factory_interface = load_interface('MeetupGroupFactory')
meetup_group_contract_interface = load_interface('MeetupGroup')
event_contract_interface = load_interface('Conference')
ens_factory_interface = load_interface('EnsSubdomainFactory')

In [ ]:
meetup_group_factory_contract = W3.eth.contract(address=MEETUP_GROUP_FACTORY_ADDRESS, abi=meetup_group_factory_interface['abi'])
ens_factory_contract = W3.eth.contract(address=ENS_FACTORY_ADDRESS, abi=ens_factory_interface['abi'])

In [ ]:
ens_factory_contract.functions.owner().call() == W3.eth.accounts[0]

In [ ]:
tx_hash = meetup_group_factory_contract.functions.createGroup(
    'YogaBerlin', 
    'Germany',
    'Berlin',
    'Sports',
    'We do cool yoga sessions',
    Web3.toBytes(text='logo')
).transact({'from': W3.eth.accounts[0], 'gas': 6200000})
tx_receipt = W3.eth.waitForTransactionReceipt(tx_hash)

In [ ]:
group_contract_address = meetup_group_factory_contract.functions.groups(0).call()

In [ ]:
group_contract = W3.eth.contract(address=group_contract_address, abi=meetup_group_contract_interface['abi'])

In [ ]:
group_contract.functions.joinGroup().transact({'from': W3.eth.accounts[1], 'gas': 6200000})

In [ ]:
tx_hash = group_contract.functions.createEvent(
    'YogaBerlin1',
    Web3.toWei(0.001, 'ether'),
    50,
    100,
    'Test',
    12345,
    Web3.toBytes(1)
).transact({'from': W3.eth.accounts[0], 'gas': 6200000})
tx_receipt = W3.eth.waitForTransactionReceipt(tx_hash)

In [ ]:
event_contract_address = group_contract.functions.events(0).call()

In [ ]:
event_contract = W3.eth.contract(address=event_contract_address, abi=event_contract_interface['abi'])

In [ ]:
event_contract.functions.register().transact({'from': W3.eth.accounts[1], 'value': Web3.toWei(0.001, 'ether')})